In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk

In [2]:
print(f"Tensorflow version              : {tf.__version__}")
print(f"Pandas version                  : {pd.__version__}")
print(f"Sklearn version                 : {sk.__version__}")
print(f"Numpy version                   : {np.__version__}")

Tensorflow version              : 2.8.0
Pandas version                  : 1.4.1
Sklearn version                 : 1.0.2
Numpy version                   : 1.22.2


In [8]:
csvFile_path = "../Dataset/pedigree dataset.csv"
collumnName = [
    "Base Color Male",
    "With White Male",
    "Base Color Female",
    "With White Female",
    "Color Prediction"
]

categorical_feature_keys = [
    "Base Color Male",
    "Base Color Female"
]

numeric_feature_keys = [
    "With White Male",
    "With White Female"
]

label_key = "Color Prediction"

In [10]:
dataset = pd.read_csv(csvFile_path, names=collumnName, delimiter=",", header=0)

dataset.head(12)

,Base Color Male,With White Male,Base Color Female,With White Female,Color Prediction
0,black,0,black,0,black
1,black,1,black,0,black
2,black,0,black,1,black
3,black,1,black,1,black
4,grey,0,grey,0,grey
5,grey,1,grey,0,grey
6,grey,0,grey,1,grey
7,grey,1,grey,1,grey
8,orange,0,orange,0,orange
9,orange,1,orange,0,orange


In [11]:
dataset.dtypes

Base Color Male      object
With White Male       int64
Base Color Female    object
With White Female     int64
Color Prediction     object
dtype: object

In [12]:
dataset.loc[2:5]

,Base Color Male,With White Male,Base Color Female,With White Female,Color Prediction
2,black,0,black,1,black
3,black,1,black,1,black
4,grey,0,grey,0,grey
5,grey,1,grey,0,grey


In [13]:
train_set, val_set = sk.model_selection.train_test_split(dataset, test_size=0.2)

AttributeError: module 'sklearn' has no attribute 'model_selection'